Este proyecto implementa un pipeline completo de machine learning orientado a predecir el rendimiento académico global de estudiantes, usando un enfoque supervisado basado en árboles de decisión.

🔍 Preprocesamiento de Datos
El preprocesamiento de los datos incluyó:

Imputación de valores faltantes:

Variables categóricas: con la moda.

Variables numéricas: con la mediana.

Codificación de variables binarias: mediante mapeo directo de "Sí/No" a 1/0.

Conversión del target categórico a valores numéricos ordenados, según el nivel de rendimiento ("bajo", "medio-bajo", "medio-alto", "alto").

Normalización y codificación adicional mediante ColumnTransformer con StandardScaler para numéricas y OneHotEncoder para categóricas (en el pipeline final).

🤖 Modelo de Machine Learning
El modelo principal utilizado fue un Random Forest Classifier, elegido por su robustez ante datos mixtos y su capacidad de manejo de no linealidades. Se implementó una validación cruzada para estimar el rendimiento y prevenir sobreajuste.

El rendimiento fue evaluado mediante accuracy, f1-score y matriz de confusión, y el mejor modelo fue persistido para su reutilización.

In [ ]:
# Importar librerías necesarias para manejo de datos y creación de pipeline de aprendizaje automático.
import pandas as pd
import numpy as np

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [7]:
# Cargar archivos de datos de entrenamiento y prueba usando pandas
# Cargar archivos
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.sample(frac=0.9, random_state=42).reset_index(drop=True)

In [8]:
# Definir función clean_data para limpieza de datos: imputación de nulos, mapeo de binarias y del target
# 2. Función de limpieza y mapeo del target
rendimiento_mapping = {
    'bajo':       1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto':       4
}

def clean_data(df, is_train=True):
    df = df.copy()
    # Imputar nulos en categóricas
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].fillna(df[col].mode()[0])
    # Mapear binarias si existen
    bin_cols = [
        'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL',
        'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET.1',
        'ESTU_PRIVADO_LIBERTAD', 'ESTU_PAGOMATRICAPROPIO'
    ]
    for col in bin_cols:
        if col in df.columns:
            df[col] = df[col].map({'Si': 1, 'No': 0, 'S': 1, 'N': 0})
    # Mapear target en train
    if is_train:
        df['RENDIMIENTO_GLOBAL_NUM'] = df['RENDIMIENTO_GLOBAL'].map(rendimiento_mapping)
    return df

df_train = clean_data(train, is_train=True)
df_test  = clean_data(test,  is_train=False)

# 3. Preparar X e y
X = df_train.drop(columns=['ID', 'RENDIMIENTO_GLOBAL', 'RENDIMIENTO_GLOBAL_NUM'])
y = df_train['RENDIMIENTO_GLOBAL_NUM']

# 4. Preprocesamiento
cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

preprocessor = ColumnTransformer([
    ("num", StandardScaler(),           num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
])

# 5. Pipeline con RandomForest
pipeline = Pipeline([
    ("prep", preprocessor),
    ("clf",  RandomForestClassifier(class_weight="balanced", random_state=42))
])


In [ ]:
# Establecer rango de hiperparámetros y configurar RandomizedSearchCV para búsqueda aleatoria
# 6. Parámetros a muestrear
param_dist = {
    'clf__n_estimators':       [300,200,100],
    'clf__max_depth':          [100,50,10],
    'clf__min_samples_split':  [10,5,1],
}

# 7. Configurar RandomizedSearchCV
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=1,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# 8. Ajustar el modelo
random_search.fit(X, y)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


RandomizedSearchCV(cv=StratifiedKFold(n_splits=2, random_state=42, shuffle=True),
                   estimator=Pipeline(steps=[('prep',
                                              ColumnTransformer(transformers=[('num',
                                                                               StandardScaler(),
                                                                               ['PERIODO',
                                                                                'FAMI_TIENEINTERNET',
                                                                                'FAMI_TIENELAVADORA',
                                                                                'FAMI_TIENEAUTOMOVIL',
                                                                                'ESTU_PRIVADO_LIBERTAD',
                                                                                'FAMI_TIENECOMPUTADOR',
                                                                                'FAMI_TIENEINTERNET.1',
                                                                                'coef_1',
                                                                                'coef_2',
                                                                                'coef_3',
                                                                                'coef_4']...
                                                                                'ESTU_HORASSEMANATRABAJA',
                                                                                'FAMI_ESTRATOVIVIENDA',
                                                                                'FAMI_EDUCACIONPADRE',
                                                                                'ESTU_PAGOMATRICULAPROPIO',
                                                                                'FAMI_EDUCACIONMADRE'])])),
                                             ('clf',
                                              RandomForestClassifier(class_weight='balanced',
                                                                     random_state=42))]),
                   n_iter=1, n_jobs=-1,
                   param_distributions={'clf__max_depth': [100],
                                        'clf__min_samples_split': [10],
                                        'clf__n_estimators': [300]},
                   random_state=42, scoring='accuracy', verbose=2)

In [11]:
# Mostrar los mejores parámetros encontrados y el accuracy del modelo
# 9. Resultados
print("Mejores parámetros:", random_search.best_params_)
print("Mejor score CV:   ", random_search.best_score_)

# 10. Informe en entrenamiento (opcional)
y_pred_train = random_search.predict(X)
print("\nClassification Report en train:")
#print(classification_report(y, y_pred_train, target_names=list(rendimiento_mapping.keys())))

Mejores parámetros: {'clf__n_estimators': 300, 'clf__min_samples_split': 10, 'clf__max_depth': 100}
Mejor score CV:    0.4152202166064982

Classification Report en train:


In [6]:
# Generar archivo de submission con las predicciones del mejor modelo
# 11. Generar archivo de submission
X_test = df_test.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'], errors='ignore')
y_test = random_search.predict(X_test)
inv_map = {v: k for k, v in rendimiento_mapping.items()}
y_labels = [inv_map[val] for val in y_test]

submission = pd.DataFrame({
    "ID": test["ID"],
    "RENDIMIENTO_GLOBAL": y_labels
})
submission.to_csv("submission_random_search2_2.csv", index=False)
print("\n📄 Archivo 'submission_random_search2_2.csv' generado.")


📄 Archivo 'submission_random_search2_2.csv' generado.
